# Analysis of healthy HP brain slicespec datasets for computation of temperature

* written by Luca Nagel (luca.nagel@tum.de) and Wolfgang Gottwald (wolfgang.gottwald@tum.de)

# 1. Importing packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib
#matplotlib.rcParams.update({'font.size': 12,'font.family':'serif','font.serif':['Computer Modern']})
from tqdm.auto import tqdm
import ipywidgets as widgets
import datetime
import pandas as pd
import os
import hypermri
import pydicom

from scipy.optimize import curve_fit
import scipy.io as sio
from matplotlib import cm

import hypermri.utils.utils_general as utg
import hypermri.utils.utils_spectroscopy as uts
import hypermri.utils.utils_fitting as utf

from hypermri.utils.utils_fitting import def_fit_params, fit_data_pseudo_inv
from hypermri.utils.utils_general import get_gmr
from hypermri.utils.utils_spectroscopy import get_metab_cs_ppm, make_NDspec_6Dspec


import sys
# define paths:
sys.path.append('../')
import Template_utsw
basepath,savepath = Template_utsw.import_all_packages()

# Autoreload extension so that you dont have to reload the kernel every time something is changed in the hypermri package
%load_ext autoreload
%autoreload 2

%matplotlib widget
def get_colors_from_cmap(cmap_name, N):
    cmap = plt.get_cmap(cmap_name)
    colors = cmap(np.linspace(0, 1, N))
    return colors

In [ ]:
error=3
study_ids=['','','','']
injections=[[1,],[1,2],[1,2],[1,2]]
fit_reps=range(0,40)
calibration_type='5mM'
all_temperatures=[]

In [ ]:
for index in tqdm(range(4)):
    study_ID=study_ids[index]
    for injection in injections[index]:
        print('Looking at',study_ID,', injection',injection)
        spectral_data = np.squeeze(sio.loadmat(os.path.join(basepath,str(study_ID)+'_raw_'+str(injection)+'.mat'))['data'])
        fid=np.squeeze(sio.loadmat(os.path.join(basepath,str(study_ID)+'_raw_'+str(injection)+'.mat'))['fid'])
        time=np.squeeze(sio.loadmat(os.path.join(basepath,str(study_ID)+'_raw_'+str(injection)+'.mat'))['time'])
        ppm = np.array(np.squeeze(sio.loadmat(os.path.join(basepath,str(study_ID)+'_raw_'+str(injection)+'.mat'))['ppm']))
        bw=5000 #Hz
        dwelltime=1/bw
        nsample_points=fid.shape[0]
        freq_range_Hz = uts.get_freq_axis(unit="Hz", sampling_dt=dwelltime, npoints=nsample_points)
        gmr = get_gmr(nucleus="13c")
        freq_range_ppm = freq_range_Hz / gmr / 3 + ppm[4095]
        #print(dwelltime,nsample_points)
        #print(f"fid.shape ={fid.shape}")
        input_fid= make_NDspec_6Dspec(input_data=fid, provided_dims=["spec", "reps", "chan","z"])
        input_spec=np.fft.fftshift(np.fft.fft(input_fid,axis=0),axes=(0,))
        #print(f"input_fid.shape={input_fid.shape}")

        mod_fid = np.conj(np.flip(input_fid,axis=0))
        mod_spec=np.flip(np.fft.fftshift(np.fft.fft(mod_fid,axis=0),axes=(0,)),axis=0)
        #print(f"input_spec.shape ={input_fid.shape}")
        #print(f"mod_fid.shape ={mod_fid.shape}")
        all_files_in_dir = os.listdir(os.path.join(basepath)+'fit_results_new_2025/')
        fitted_files = []
        for index,file in enumerate(all_files_in_dir):
            if file.endswith('.pkl'):
                fitted_files.append(file)
                #print('Found files ',file)
        # sort files by number
        fitted_files.sort()
        # look for file matching the studyfolder_num
        for n in range(len(fitted_files)):
            if fitted_files[n].find(str(study_ID)+'_fit_spectra_study_'+str(injection))==0:
                load_file=fitted_files[n]
            else:
                pass

        print('Loading',load_file)
        fit_results=utg.load_as_pkl(dir_path=os.path.join(basepath, 'fit_results_new_2025/'), filename=load_file, global_vars=globals())
        fit_spectrums = fit_results['fit_spectrums']
        fit_amps = fit_results['fit_amps']
        fit_freqs = fit_results['fit_freqs']
        fit_t2s = fit_results['fit_t2s']
        fit_params = fit_results['fit_params']
        fit_stds=fit_results['fit_stds']

        # Frequency difference [Hz]:

        freq_diff_hz = np.take(fit_freqs, indices=1, axis=-1) - np.take(fit_freqs, indices=0, axis=-1)

        # Frequency difference [ppm]:
        freq_diff_ppm = uts.freq_Hz_to_ppm(freq_Hz  = freq_diff_hz,
                                           hz_axis  = fit_params["freq_range_Hz"],
                                           ppm_axis = fit_params['freq_range_ppm'],
                                           ppm_centered_at_0=True)

        # Temperature [Hz]:
        temp, _ = utf.temperature_from_frequency(frequency=freq_diff_ppm,
                                                 calibration_type=calibration_type,
                                                 frequency_is_ppm=True)

        # std[freq difference [Hz]] [0/1 = pyr/lac, 1=freqs]
        freq_diff_hz_std = np.sqrt(fit_stds[..., 0, 1]**2 +
                                   fit_stds[..., 1, 1]**2 )

        freq_diff_ppm_std = uts.freq_Hz_to_ppm(freq_Hz=freq_diff_hz_std,
                                           hz_axis  = fit_params["freq_range_Hz"],
                                           ppm_axis = fit_params['freq_range_ppm'],
                                           ppm_centered_at_0=True)


        temp_plus_std, _ = utf.temperature_from_frequency(frequency=freq_diff_ppm+freq_diff_ppm_std,
                                                 calibration_type=calibration_type,
                                                 frequency_is_ppm=True)

        temp_std = np.abs(temp_plus_std - temp)

        temperature_all=temp[0,:,0,0,fit_reps,:]
        dtemperature_all=temp_std[0,:,0,0,fit_reps,:]
        
        temp_error_mask=np.where(dtemperature_all <= error, 1, np.nan)
        all_temperatures.append(np.where(~np.isnan(temp_error_mask), temperature_all, np.nan))
    
        plt.close('all')
        colors = get_colors_from_cmap('tab10', 10)
        fig,ax=plt.subplots(2,2,figsize=(7,5),tight_layout=True)
        mean_per_slice,std_per_slice,num_per_slice=[],[],[]
        for n in range(4):
            nx=n//2
            ny=n%2
            foo=[]
            dfoo=[]
            nfoo=0
            for chan in range(8):
                fit_reps=np.array(fit_reps)
                idx=np.squeeze(np.where(dtemperature_all[:,n,chan]<error))
                ax[nx,ny].plot(fit_reps[idx],temperature_all[idx,n,chan],'-o',markersize=3,color=colors[chan],label='chan '+str(chan))
                foo.append(np.nanmean(temperature_all[idx,n,chan]))
                dfoo.append(np.nanmean(temperature_all[idx,n,chan]))
                nfoo+=(len(np.ravel(temperature_all[idx,n,chan])))
                ax[nx,ny].set_ylim([20,45])

                ax[nx,ny].set_xlabel('Repetition')
                ax[nx,ny].set_ylabel(r'T[$^\circ$C]')

            mean_per_slice.append(np.nanmean(foo))
            std_per_slice.append(np.nanstd(dfoo))
            num_per_slice.append(nfoo)
        ax[0,1].legend(ncol=3,fontsize=8)

        fig.suptitle('Temperature per slice')
        for n in range(4):
            nx=n//2
            ny=n%2
            ax[nx,ny].hlines(mean_per_slice[n],5,25,linestyle='dashed',color='k',alpha=0.45)
            ax[nx,ny].set_title('Slice '+str(n)+', T='+str(np.round(mean_per_slice[n],1))+r"$^\circ$")



        df_output=pd.DataFrame(columns=['ID','slice','T','dT','n'])
        rows=[]
        for slic in range(4):

            row={
                'ID': study_ID,
                'inj':injection,
                'slice': slic,
                'T': mean_per_slice[slic],
                'dT': std_per_slice[slic],
                'n': num_per_slice[slic]}
            rows.append(row)    
        df_output = pd.concat([df_output, pd.DataFrame(rows)], ignore_index=True)

